In [1]:
import os,sys
sys.path.insert(0, os.path.abspath(os.getcwd() + "/../build/python/"))
import pyqedfv as qedfv
import numpy as np
from scipy.optimize import brentq

In [2]:
# from spherical to Cartesian coordinates
def toCartesian(v):
    x = v[0] * np.sin(v[1]) * np.cos(v[2])
    y = v[0] * np.sin(v[1]) * np.sin(v[2])
    z = v[0] * np.cos(v[1])
    return np.array([x, y, z])

In [3]:
# inputs
vnorm = 0.99
j = 0.
qedr = True

# defining qedl and qedr
cqedl = qedfv.Coef()
cqedr = qedfv.Coef(qed=qedfv.Qed.r)

pv = np.array([vnorm, 0.0, 0.0])

if qedr == True:
    par = cqedr.tune(j, pv)
    f = lambda th, phi: cqedr(j, toCartesian([vnorm, th, phi]), par)
    filename = "../tests/roots_c%d_v%s_qedr.txt" % (int(j), vnorm)
else:
    par = cqedl.tune(j, pv)
    f = lambda th, phi: cqedl(j, toCartesian([vnorm, th, phi]), par)
    filename = "../tests/roots_c%d_v%s_qedl.txt" % (int(j), vnorm)

# finding roots
results = []

for phi in np.arange(0, np.pi / 4.0, 0.015):
    sphi = np.sin(phi)
    thMin = -2.0 * np.arctan(sphi - np.sqrt(1.0 + sphi ** 2.0))
    thMax = np.pi / 2.0
    thHalf = thMin + 0.5 * (thMax - thMin)

    if f(thMin, phi) * f(thHalf, phi) < 0:
        try:
            th = brentq(f, thMin, thHalf, args=(phi,))
            results.append([th, phi, f(th, phi)])
        except ValueError as e:
            print("Error:", e)
    
    if f(thHalf, phi) * f(thMax, phi) < 0:
        try:
            th = brentq(f, thHalf, thMax, args=(phi,))
            results.append([th, phi, f(th, phi)])
        except ValueError as e:
            print("Error:", e)

# print on file
np.savetxt(filename, results, fmt='%.18e')